In [ ]:
import maup # mggg's library for proration, see documentation here: https://github.com/mggg/maup
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

In [ ]:
vest_mo_18 = gp.read_file("./raw-from-source/VEST/mo_2018/mo_2018.shp")

## Election Data Sources

Election results from the Secretary of State's office via OpenElections (https://github.com/openelections/openelections-data-mo/tree/master/2018). Absentee, provisional, and mail ballots were reported countywide in nearly every county; these were distributed by candidate to precincts based on their share of the precinct-level reported vote.

- G18USSRHAW - Josh Hawley (Republican Party)  
- G18USSDMCC - Claire McCaskill (Democratic Party)  
- G18USSLCAM - Japheth Campbell (Libertarian Party)  
- G18USSGCRA - Jo Crain (Green Party)  
- G18USSIODE - Craig O'Dear (Independent)  
  
  
- G18AUDRMCD - Saundra McDowell (Republican Party)  
- G18AUDDGAL - Nicole Galloway (Democratic Party)  
- G18AUDLOTO - Sean O'Toole (Libertarian Party)  
- G18AUDGFIT - Don Fitz (Green Party)  
- G18AUDCLUE - Jacob Luetkemeyer (Constitution Party)  


In [ ]:
mo_elections = pd.read_csv("./raw-from-source/Open_Elections/20181106__mo__general__precinct.csv")
candidate_list = ['Josh Hawley','Claire McCaskill','Japheth Campbell','Jo Crain',
 "Craig O'Dear",'Saundra McDowell','Nicole Galloway',"Sean O'Toole",'Don Fitz','Jacob Luetkemeyer']
office_list = ['U.S. Senate', 'State Auditor']
mo_elections["candidate"] = mo_elections["candidate"].str.strip()
mo_elections = mo_elections[mo_elections["office"].isin(office_list)]
mo_elections = mo_elections[mo_elections["candidate"].isin(candidate_list)]


In [ ]:
fips_file = pd.read_csv("./raw-from-source/FIPS/US_FIPS_Codes.csv")
fips_file = fips_file[fips_file["State"]=="Missouri"]
fips_file["FIPS County"] = fips_file["FIPS County"].astype(str)
fips_file["FIPS County"] = fips_file["FIPS County"].str.zfill(3)
#print(fips_file[~fips_file["County Name"].isin(mo_elections["county"])]["County Name"].unique())
fips_dict = dict(zip(fips_file['County Name'], fips_file['FIPS County']))
mo_changes = {'De Kalb':'Dekalb','Kansas City':'Jackson','McDonald':'Mcdonald',
              'St. Charles':'St Charles','St. Clair':'St Clair',
              'Ste. Genevieve':'Ste Genevieve','St. Francois':'St Francois',
              'St. Louis City':'St Louis City','St. Louis County':'St Louis'}
mo_elections['county'] = mo_elections['county'].map(mo_changes).fillna(mo_elections['county'])
mo_elections['COUNTYFP'] = mo_elections['county'].map(fips_dict).fillna(mo_elections['county'])
mo_elections["unique_ID"]=mo_elections["COUNTYFP"]+mo_elections["precinct"]

In [ ]:
mo_elections_pivot = pd.pivot_table(mo_elections,index=["unique_ID","COUNTYFP","precinct"],columns=["candidate"],values=['votes'],aggfunc=sum)
mo_elections_pivot = mo_elections_pivot.fillna(0)
mo_elections_pivot.columns = mo_elections_pivot.columns.droplevel(0)
mo_elections_pivot.reset_index(drop=False,inplace=True)
mo_elections_pivot = mo_elections_pivot.rename(columns={'Claire McCaskill':"G18USSDMCC","Craig O'Dear": "G18USSIODE",'Don Fitz':"G18AUDGFIT",'Jacob Luetkemeyer':"G18AUDCLUE",'Japheth Campbell':"G18USSLCAM",'Jo Crain':"G18USSGCRA",'Josh Hawley':"G18USSRHAW",'Nicole Galloway':"G18AUDDGAL",'Saundra McDowell':"G18AUDRMCD","Sean O'Toole":"G18AUDLOTO"})

In [ ]:
remove_list = ["ABSENTEE","PROVISIONAL","FEDERAL","FEDERAL/STATE","INTRASTATE","WRITE-IN","CENTRAL POLLING LOCATION","CENTRAL POLL","ABSENTEE1","ABSENTEE2","VOTE CENTER","CENTRAL POLLING LOCATION: MSU","INTRASTATE/NEW RESIDENT","WRITE-INS","MULTI-PRECINCT COLLECTION","INTRASTATE NEW RESIDENT","MULTI-PRECINCT COLLECTION","ABSENTEE, PROVISIONAL & CENTRAL"]
mo_elections_countywide = mo_elections_pivot[(mo_elections_pivot["precinct"].isin(remove_list)) | (mo_elections_pivot["unique_ID"]=='019CENTRAL')]
mo_elections_pivot = mo_elections_pivot[~(mo_elections_pivot["precinct"].isin(remove_list)) | (mo_elections_pivot["unique_ID"]=='019CENTRAL')]
print(mo_elections_countywide.head())
print(mo_elections_pivot.head())
countywide_totals = mo_elections_countywide.groupby(["COUNTYFP"]).sum()
precinct_totals = mo_elections_pivot.groupby(["COUNTYFP"]).sum()

In [ ]:
data_to_add = pd.merge(precinct_totals,countywide_totals,on=['COUNTYFP'],how='left',indicator=False)
data_to_add.reset_index(inplace=True,drop=False)

joined = pd.merge(mo_elections_pivot,data_to_add,on=['COUNTYFP'],how='left',indicator=False)
joined = joined.fillna(0)
col_list = ["G18USSDMCC","G18USSIODE","G18AUDGFIT","G18AUDCLUE","G18USSLCAM","G18USSGCRA","G18USSRHAW","G18AUDDGAL","G18AUDRMCD","G18AUDLOTO"]
for i in col_list:
    precinct_totals = i + "_x"
    countywide_reported = i+"_y"
    joined[i]=(joined[i]+((joined[i]*joined[countywide_reported])/joined[precinct_totals]))
joined = joined [["unique_ID","COUNTYFP","precinct","G18USSDMCC","G18USSIODE","G18AUDGFIT","G18AUDCLUE","G18USSLCAM","G18USSGCRA","G18USSRHAW","G18AUDDGAL","G18AUDRMCD","G18AUDLOTO"]]
matcher_increase = pd.read_csv("./matcher_increase.csv")
print(matcher_increase.head())
new_name_dict = dict(zip(matcher_increase['source_names'], matcher_increase['vest_names']))
joined["unique_ID"] = joined["unique_ID"].str.strip()
joined["unique_ID"] = joined["unique_ID"].map(new_name_dict).fillna(joined["unique_ID"])

- G18USSRHAW - Josh Hawley (Republican Party)  
- G18USSDMCC - Claire McCaskill (Democratic Party)  
- G18USSLCAM - Japheth Campbell (Libertarian Party)  
- G18USSGCRA - Jo Crain (Green Party)  
- G18USSIODE - Craig O'Dear (Independent)  
  
  
- G18AUDRMCD - Saundra McDowell (Republican Party)  
- G18AUDDGAL - Nicole Galloway (Democratic Party)  
- G18AUDLOTO - Sean O'Toole (Libertarian Party)  
- G18AUDGFIT - Don Fitz (Green Party)  
- G18AUDCLUE - Jacob Luetkemeyer (Constitution Party)  

In [ ]:
vest_mo_18["unique_ID"]=vest_mo_18["COUNTYFP"]+vest_mo_18["NAME"]
vest_empty = vest_mo_18[(vest_mo_18['G18USSRHAW']==0) & (vest_mo_18['G18USSDMCC']==0) &
              (vest_mo_18['G18USSLCAM']==0) & (vest_mo_18['G18USSGCRA']==0) &
              (vest_mo_18['G18USSIODE']==0) & (vest_mo_18['G18AUDRMCD']==0) &
              (vest_mo_18['G18AUDDGAL']==0) & (vest_mo_18['G18AUDLOTO']==0) &
              (vest_mo_18['G18AUDGFIT']==0) & (vest_mo_18['G18AUDCLUE']==0)]["unique_ID"]

new_precincts = pd.DataFrame(columns=joined.columns)
new_precincts["unique_ID"]=vest_empty
col_list = ["G18USSDMCC","G18USSIODE","G18AUDGFIT","G18AUDCLUE","G18USSLCAM","G18USSGCRA","G18USSRHAW","G18AUDDGAL","G18AUDRMCD","G18AUDLOTO"]
for i in col_list:
    new_precincts[i]=0
joined = joined.append(new_precincts)

In [ ]:
print(vest_mo_18.head())

final_join = pd.merge(vest_mo_18,joined,how="outer",on="unique_ID",indicator=True)
print(final_join["_merge"].value_counts())
final_join[final_join["_merge"]=="right_only"].to_csv("./election_only.csv")
final_join[final_join["_merge"]=="left_only"].to_csv("./VEST_only.csv")

## Precinct Shapefile Sources

The documentation mentions 4 different sources for shapefiles:

1. Precinct shapefiles primarily from the U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release.  
2. Monroe County comes from the 2010 Census VTD release.  
3. Platte County comes from the 2020 Census VTD release.  
4. The following counties instead used maps or shapefiles sourced from the respective county governments: Audrain, Bates, Caldwell, Callaway, Camden, Cape Girardeau, Cedar, Cooper, Franklin, Greene, Jackson, Jasper, Jefferson, Laclede, Lafayette, Lawrence, Marion, McDonald, Miller, Nodaway, Osage, Ozark, Pemiscot, Pike, Randolph, Scott, Ste. Genevieve, Texas, Warren, Washington, Worth, Wright.  

### 1. U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release
> Precinct shapefiles primarily from the U.S. Census Bureau's 2020 Redistricting Data Program Phase 2 release.

https://www.census.gov/geo/partnerships/pvs/partnership19v2/st29_mo.html

In [ ]:
#When downloading from the Census redistricing data program, these use a FIPS code to identify counties

fips_file["unique_ID"] =  "29" + fips_file["FIPS County"]
fips_codes = fips_file["unique_ID"].tolist()

#Combine all the data from separate files into one
li = []
for i in fips_codes:
    ref = "./raw-from-source/Census/2020_Phase_2/partnership_shapefiles_19v2_"
    file_ref = ref+i+"/PVS_19_v2_vtd_"+i+".shp"
    file_prev = gp.read_file(file_ref)
    #print(file_prev.shape)
    li.append(file_prev)
shapefiles_census = pd.concat(li, axis=0, ignore_index=True)
print(shapefiles_census.shape)

In [ ]:
#print(shapefiles_census.head())
county_list = ["Audrain", "Bates","Caldwell","Callaway","Camden","Cape Girardeau","Cedar","Cooper","Franklin","Greene","Jackson","Jasper","Jefferson", 
               "Laclede", "Lafayette","Lawrence", "Marion", "Mcdonald","Miller","Monroe","Nodaway",
               "Osage", "Ozark", "Pemiscot","Platte", "Pike", "Randolph", "Scott","Ste Genevieve", "Texas", "Warren", "Washington", "Worth", "Wright"]
print(len(county_list))
fips_to_take = fips_file[fips_file["County Name"].isin(county_list)]
different_source = fips_to_take["FIPS County"].tolist()
print(different_source)

shapefiles_census = shapefiles_census[~shapefiles_census["COUNTYFP"].isin(different_source)]

In [ ]:
#Filter down election results to just these counties
print(joined.shape)
elections_census_shapes = joined[~joined["COUNTYFP"].isin(different_source)]
print(elections_census_shapes.shape)

#Create a unique ID
shapefiles_census["unique_ID"] = shapefiles_census["COUNTYFP"]+shapefiles_census["NAME"]

#Attempt to join on the "unique_ID" column
census_join = pd.merge(elections_census_shapes,shapefiles_census,how="outer",on="unique_ID",indicator=True)
print(census_join["_merge"].value_counts())
census_join[census_join["_merge"]=="left_only"].to_csv("./census_join/election.csv")
census_join[census_join["_merge"]=="right_only"].to_csv("./census_join/shapes.csv")

In [ ]:
print(shapefiles_census[shapefiles_census["COUNTYFP"].str[0:3]=="009"]["unique_ID"])
print(elections_census_shapes[elections_census_shapes["unique_ID"].str[0:3]=="009"]["unique_ID"])

In [ ]:
precinct_merger(["003John Glenn A" "003Country Club","003John Glenn","003Beeler"],"003John Glenn/Country Club/Beeler")
precinct_merger(["003Platte West","003Rosendale","003Bolckow"],"003Platte West/Rosendale/Bolckow")
precinct_merger(["003Fillmore","003Clay"],"003Fillmore/Clay")
precinct_merger(["003Empire East","003Platte East"],"003Platte East/Empire East")
precinct_merger(["003Savannah Northeast","003Savannah Southeast"],"003Savannah Northeast/Southeast")
precinct_merger(["003Savannah Northwest","003Savannah Southwest"],"003Savannah Northwest/Southwest")
precinct_merger(["009McDonald","009Butterfield"],"009McDonald/Butterfield")
precinct_merger(["009Cassville Rural","009Mineral"],"009Cassville Rural/Mineral")
precinct_merger(["009Kings Prairie","009Pleasant Ridge"],"009Kings Prairie/Pleasant Ridge")
precinct_merger(["009Capps Creek"],"009Monett/Capps Creek")
precinct_merger(["009Crane Creek","009Ozark"],"009Ozark/Crane Creek")
precinct_merger(["009Purdy","009McDowell"],"009McDowell/Purdy")
precinct_merger(["009Ash","009Washburn"],"009Washburn/Ash")
precinct_merger(["015COLE"],"015Cole")
precinct_merger(["015ALEXANDER","015FRISTOE"],"015Fristoe/Alexander")
precinct_merger(["015LINDSEY"],"015Lindsey")
precinct_merger(["015TOM"],"015Tom")
precinct_merger(["015UNION"],"015Union")
precinct_merger(["015WHITE"],"015White")
precinct_merger(["015WILLIAMS"],"015Williams")

precinct_merger(["017Drum-Dongola"],"017Drum Dungola")
precinct_merger(["017Lutesville","017Marble Hill"],"017Lutesville/Marble Hill")

county_list = ["01909/11","01910/12","01915/40","01918/19","0191C/1G","0191E/6B","0191F/1J/1L","0191I/6F","0191K/2K","01922/45","01924/33","01925/34","01928/29","0192A/2G","0192B/2J","0192C/2F","0192L/20","01930/43","01931/32","01935/42","0193A/3G/3L","0193E/3H","0193I/3J","0193M/6J","0193N/41","0194A/4J","0194C/4D","0194F/4I/5A","0194H/4N","0194K/4M/37","0195C/5H","0195D/5G/5M","0195E/5P","0195I/5J","0195N/36/44","0196A/6L","0196C/6H","0196D/6K/6M","0196E/6I"]
#For those, split on the "/" and call precinct merger
county_list_2 = pd.read_csv("./189_columns.csv")
county_list_3 = list(county_list_2["val"])
print(county_list_3)


In [ ]:
txt = "0191F/1J/1L"
txt.split("/")

Missouri
--------

Precinct mergers were made in the following counties to match county reporting units: 
- Andrew: Precinct mergers to match county reporting units
- Barry: Precinct mergers to match county reporting units
- Barry: Add Monett City; Align Cassville with corporate boundary
- Barton: Adjust Lamar ward boundaries to match city map
- Benton: Precinct mergers to match county reporting units
- Bollinger: Precinct mergers to match county reporting units
- Bollinger: Adjust Patton/Sedgewickville to match county GIS
- Butler: Align Poplar Bluff with corporate boundary and adjust wards to match city map
- Boone: Precinct mergers to match county reporting units
- Carroll: Precinct mergers to match county reporting units
- Carroll: Adjust Carrollton City wards to match voter file
- Carter: Precinct mergers to match county reporting units
- Cass: Precinct mergers to match county reporting units
- Cass: Align Belton, Pleasant Hill, Raymore precincts with corporate boundaries; Adjust 37 Pleasant Prairie/25 West Peculiar Rural
- Chariton: Adjust Salisbury wards to match city map
- Christian: Revise Cassidy 1/Cassidy 2; Reverse Nixa City annexation into Rosedale 1B
- Christian: Precinct mergers to match county reporting units
- Clark: Precinct mergers to match county reporting units
- Clay: Adjust Excelsior Springs, Kearney, Smithville precincts to match 2018 ward and corporate boundaries; Adjust KC 21-1/15, 21-1/17, 21-3/11, 21-4/6, 21-4/11, 21-5/7/24, 21-6/24, 21-7/20, 21-13/26, 21-14/23, 21-17/16, 21-17/26, 21-17/Platte 1, Gallatin 7/16, 10/13, 10/16, 11/15, 13/15, Kearney 3/Platte 2, Liberty 1/10, 6/13, 7/11, 11/14, Washington 1/3 to match voter file
- Clinton: Align Atchison/Concord, Jackson/Lathrop with 2018 precinct split assignments
- Cole: Precinct mergers to match county reporting units
- Cole: Adjust Marion Twp St Martins to include St Martins city in Jefferson Twp
- Crawford: Move Benton Twp section of Leasburg precinct from Recklein to Leasburg
- Crawford: Precinct mergers to match county reporting units
- Dade: Realign Greenfield wards to match municipal code
- Dallas: Split N Benton Inside/OCL, S Benton Inside/OCL; Align N/S Benton with Buffalo ward map
- Dallas: Precinct mergers to match county reporting units
- Dekalb: Precinct mergers to match county reporting units
- Dent: Split Boss precinct from R-4 precinct; Adjust R-2/R-3 boundary
- Douglas: Precinct mergers to match county reporting units
- Dunklin: Precinct mergers to match county reporting units
- Dunklin: Align Holcomb, Rives with township boundaries; Adjust Arbyrd/Senath; Align Kennett wards with city map
- Gasconade: Align Hermann, Owensville, Rosebud with corporate boundaries and ward descriptions
- Gentry: Precinct mergers to match county reporting units
- Grundy: Precinct mergers to match county reporting units
- Holt: Precinct mergers to match county reporting units
- Holt: Split West between Central/Northwest by school district
- Howard: Precinct mergers to match county reporting units
- Howell: Precinct mergers to match county reporting units
- Howell: Align West Plains precincts with 2018 corporate boundary
- Iron: Adjust Annapolis/Des Arc/Vulcan, Ironton/Pilot Knob
- Johnson: Adjust Warrensburg NE/NW boundary to match county maps
- Johnson: Precinct mergers to match county reporting units
- Lewis: Split LaBelle Twp between LaBelle FS/Lewistown FS with PLSS grid
- Lincoln: Align Moscow Mills, Troy with 2018 corporate boundaries; Adjust Briscoe/Troy Rural
- Linn: Precinct mergers to match county reporting units
- Linn: Split Brookfield Township between Brookfield 1&2/3&4
- Livingston: Align Chillicothe with 2018 corporate boundaries
- Livingston: Precinct mergers to match county reporting units
- Macon: Align Macon City wards with corporate boundaries
- Madison: Align Fredericktown wards with corporate boundaries
- Mercer: Precinct mergers to match county reporting units
- Mississippi: Precinct mergers to match county reporting units
- Mississippi: Adjust Bertrand/East Prairie Rural; Align East Prairie City with corporate boundary
- Moniteau: Precinct mergers to match county reporting units
- Montgomery: Align Bellflower/Middletown, Big Spring/Rhineland with voter file
- Morgan: Adjust Versaille North/South boundary to match county GIS
- Morgan: Precinct mergers to match county reporting units
- New Madrid: Precinct mergers to match county reporting units
- New Madrid: Add New Madrid City wards; Add Big Prairie 4 precinct; Move Portageville Ward 1 to Portage 1
- Newton: Align Staples precincts with Joplin City GIS boundaries; Align Neosho precincts with Neosho City GIS boundaries; Adjust Newtonia/Ritchey Wentworth
- Oregon: Precinct mergers to match county reporting units
- Pettis: Precinct mergers to match county reporting units
- Pettis: Adjust Sedalia City wards to match city map
- Phelps: Align St. James ward boundaries with city map
- Phelps: Precinct mergers to match county reporting units
- Polk: Precinct mergers to match county reporting units
- Polk: Split Campbell/Jefferson precincts by school district
- Putnam: Split Union Twp into NE/SW precincts to match county map
- Putnam: Precinct mergers to match county reporting units
- Pulaski: Adjust Big Piney/St. Roberts, Laquey/Waynesville
- Ralls: Precinct mergers to match county reporting units
- Ralls: Replace Center/Liberty precincts with 2010 VTDs
- Ray: Adjust Richmond City wards, Homestead Village/Wood Heights/Lawson, Richmond/Orrick
- Ray: Precinct mergers to match county reporting units
- Reynolds: Realign precincts countywide to match voter file
- Ripley: Precinct mergers to match county reporting units
- Saline: Split Nelson precinct from Hardeman precinct; Adjust Marshall Ward 2/3 boundary
- St. Charles: Adjust Brian/Ridgepoint, Civic/St. Paul, Harvester/Sycamore, Mamelle/Marina
- St. Clair: Precinct mergers to match county reporting units
- St. Francois: Precinct mergers to match county reporting units
- St. Francois: Align Desloge, Farmington, Park Hills with corporate boundaries and city ward maps; Adjust Bonne Terre/Timberline, Patterson/Timberline, Leadwood/Park Hills to match voter file
- St. Louis: Precinct mergers to match county reporting units
- St. Louis: Add UNV044 precinct; Split NRW011/NRW052
- Schuyler: Precinct mergers to match county reporting units
- Shannon: Precinct mergers to match county reporting units
- Shannon: Align all precincts with county GIS and voter file
- Shelby: Precinct mergers to match county reporting units
- Shelby: Adjust Jackson/North River to match voter file
- Stoddard: Precinct mergers to match county reporting units
- Stoddard: Align Advance, Bernie, Bloomfield, Dexter, Puxico with corporate boundaries; Adjust Advance/Leora, Bernie/Dexter/Pyle, Bloomfield/Gray Ridge
- Stone: Move Branson West from Ruth A to Ruth C; Split Ruth C Rural/Sunset Cove; Adjust Ruth B City/Rural to match county GIS
- Sullivan: Precinct mergers to match county reporting units
- Taney: Adjust Boston Center/Walnut Shade; Align Branson wards with 2018 corporate boundary
- Vernon: Align Bronaugh/Moundville, Harwood/Schell City, Deerfield/NG Armory with voter file
- Wayne: Precinct mergers to match county reporting units
- Webster: Precinct mergers to match county reporting units





### 2. 2010 Census VTD release
>Monroe County comes from the 2010 Census VTD release.

https://www.census.gov/cgi-bin/geo/shapefiles/index.php?year=2010&layergroup=Voting+Districts

In [ ]:
monroe_shape = gp.read_file("./raw-from-source/Census/2010_VTD/tl_2010_29137_vtd10/tl_2010_29137_vtd10.shp")

### 3. 2020 Census VTD release
>Platte County comes from the 2020 Census VTD release.

https://www.census.gov/geographies/mapping-files/time-series/geo/tiger-line-file.2020.html

In [ ]:
platte_shape = gp.read_file("./raw-from-source/Census/2020_VTD/tl_2020_29_all/tl_2020_29165_vtd20.shp")
platte_shape.plot()

- Platte: Precinct mergers to match county reporting units
- Platte: Align Lake Waukomis, Parkville, Platte City, Riverside/Northmore, Weatherby Lake with corporate boundaries; Adjust Platte Hills/Park Hill, Seven Bridges/Shiloh

### 4. Counties themselves
> The following counties instead used maps or shapefiles sourced from the respective county governments:  
Audrain, Bates, Caldwell, Callaway, Camden, Cape Girardeau, Cedar, Cooper, Franklin, Greene, Jackson, Jasper, Jefferson, Laclede, Lafayette, Lawrence, Marion, McDonald, Miller, Nodaway, Osage, Ozark, Pemiscot, Pike, Randolph, Scott, Ste. Genevieve, Texas, Warren, Washington, Worth, Wright.  

- Audrain: https://audraingis.integritygis.com/H5/Index.html?viewer=audrain
- Bates: https://batesgis.integritygis.com/H5/Index.html?viewer=bates
- Caldwell: can't find
- Callaway: can't find
- Camden: https://camdengis.integritygis.com/H5/Index.html?viewer=camden
- Cape Girardeau: only pdf (https://capecountyelections.com/wp-content/uploads/2019/06/Voter_Precincts.pdf)
- Cedar: can't find
- Cooper:
- Franklin:
- Greene:
- Jackson:
- Jasper:
- Jefferson:
- Laclede:
- Lafayette:
- Lawrence:
- Marion:
- McDonald:
- Miller:
- Nodaway:
- Osage:
- Ozark:
- Pemiscot:
- Pike:
- Randolph:
- Scott:
- Ste. Genevieve:
- Texas:
- Warren:
- Washington:
- Worth:
- Wright:

### Shapefile Modifications

- Audrain: Precinct mergers to match county reporting units
- Audrain: Add Vandalia City; Align Mexico precincts with municipal code
  
- Callaway: Align Fulton, Holts Summit precincts with 2018 corporate boundaries
- Callaway: Precinct mergers to match county reporting units

- Camden: Precinct mergers to match county reporting units

- Cape Girardeau: Precinct mergers to match county reporting units

- Cooper: Precinct mergers to match county reporting units

- Franklin: Split Dry Branch between Stanton & St. Clair Out of Town
- Franklin: Precinct mergers to match county reporting units

- Greene: Precinct mergers to match county reporting units

- Jackson: Precinct mergers to match county reporting units

- Jasper: Precinct mergers to match county reporting units
- Jasper: Realign Carthage 2/3, Joplin 1/7, 10/13, 10/15, 13/22, Oronogo 1/2, Webb City 1/4 to match city ward maps

- Jefferson: Precinct mergers to match county reporting units
- Jefferson: Split Festus Ward 1 P1/P2, Ward 3 P1/P2, Ward 4 P1/P2; Adjust Crystal City/Riverview

- Marion: Precinct mergers to match county reporting units
- Marion: Split Fabius-Smileyville/Taylor, Warren-Monroe City/Warren; Align Hannibal wards with voter file

- Miller: Adjust all precinct boundaries to match voter file

- Nodaway: Precinct mergers to match county reporting units

- Osage: Precinct mergers to match county reporting units

- Pemiscot: Precinct mergers to match county reporting units

- Pike: Adjust Cuivre JL boundaries with Ashley NO, Buffalo E, Prairieville I

- Randolph: Precinct mergers to match county reporting units

- Scott: Adjust Haywood City/Morley, Sikeston Ward 1/McMullin/Vanduser

- Warren: Precinct mergers to match county reporting units

- Washington: Adjust City Hall/Stony Point, Cruise/Richwoods to match county GIS

- Worth: Precinct mergers to match county reporting units